# Test Sever Instance
* Creators: Nirvan S P Theethira, Zach Allen McGrath
* Date: 3/6/2020
* Purpose: The server replica is built for testing the group framework in `phaseO.py`.

In [1]:
import socket
import struct
import regex as re
import threading 
import os, sys
import time
sys.path.append('/home/nirvan/Desktop/github/CSCI5673_Assignments/assignment2/phaseO')
# Threads from phaseO.py are tested.
import phaseO

In [2]:
# variables to change for each replica
# each replica should use a unique address
uniAddrPort = ("127.0.0.1", 8081)
# each replica should have a unique memeber number.
# each new member added should have an incemental member number [0,1,2].
# A group missing a member number will fail
# This is a rudementary member join protocal that will be updated in phase 2
memberNumber = 1
# Set to True for macos
IS_MACOS = False

* Code below is to bind to unicast and multicast group sockets.
* The `mutiListner` and `uniListner` threads will listen to these sockets

In [3]:
# binding to multicast group socket
# this should match the address of other group members
MCAST_GRP = '224.0.0.1'
MCAST_PORT = 5050
multiAddrPort = (MCAST_GRP, MCAST_PORT)
LOCAL_HOST = True

reuse_option = socket.SO_REUSEADDR if not IS_MACOS else socket.SO_REUSEPORT
level_option = socket.IP_ADD_MEMBERSHIP if not LOCAL_HOST else socket.IP_MULTICAST_LOOP

mcast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
mcast_sock.setsockopt(socket.SOL_SOCKET, reuse_option, 1)
mcast_sock.bind((MCAST_GRP, MCAST_PORT))
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
mcast_sock.setsockopt(socket.IPPROTO_IP, level_option, mreq)

# starting unicast
ucast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ucast_sock.bind(uniAddrPort)

* The code below starts threads from `phaseO.py` to listen to `mcast_sock` and `ucast_sock` sockets.
* It also starts the middleware thread from `phaseO.py` to process commands that come in from the sockets.

In [4]:
# buffer stores data packets received by multiThread thread and uniThread thread
commands = []

# used as flag to terminate while loop of multiThread
# thread terminates when the len of multiRunning is greater than 0
# aka insert element
multiRunning = []
# starting multiThread
multiThread = threading.Thread(target=phaseO.mutiListner, name='multicast', 
    args=(multiRunning, commands, mcast_sock, uniAddrPort))
multiThread.daemon = True
multiThread.start()

# used as flag to terminate while loop of uniThread
# thread terminates when the len of uniRunning is greater than 0
# aka insert element
uniRunning = []
# starting uniRunning
uniThread = threading.Thread(target=phaseO.uniListner, name='unicast', 
    args=(uniRunning, commands, ucast_sock, uniAddrPort))
uniThread.daemon = True
uniThread.start()

# ALL GLOBAL VARIABLES
# the global sequence number maintained by this server
globalSeq = [0]
# stores all memebers of the group
# members are added when join request confirmation is received
groupMembers = {memberNumber:uniAddrPort}
# FTQueue instance for this server
FTq = phaseO.FTQueue()
# message buffer
messages = {}
# all sequence numbers assigned/generated by this server
# used to resend sequence number when NAK is sent from other servers
seqNumsSent = {}

# TESTING VARIABLES
# list of messages to force drop 
# aka prevent messages with these seq numbers from initial being sent to other group members
# This forces the other memeber to send a NAK to this member
testLostMessages = []
# flag to prevent the message buffer from being cleared
# this prevents NAK from being sent until len of testBuildup > 0
testBuildup = []

# used as flag to terminate while loop of midThread
# thread terminates when the len of midRunning is greater than 0
# aka insert element
midRunning = []
# starting midThread
midThread = threading.Thread(target=phaseO.middlewareThread, name='middleware', 
    args=(midRunning, globalSeq, memberNumber, groupMembers, 
        FTq, commands, messages, seqNumsSent, 
        ucast_sock, multiAddrPort, testLostMessages, testBuildup))
midThread.daemon = True
midThread.start()

Running Multicast Listner
Running Unicast Listner


* checking to see if member number is unique to the group
* if a group member with this number exists request is denied another number has to choosen

In [5]:
# joining group
joinM = ('joinRequest', memberNumber)
joinM = ",".join(map(lambda x: str(x),joinM))
ucast_sock.sendto(str.encode(joinM), (MCAST_GRP, MCAST_PORT))

13

Received ['joinConfirm', '0'] from ('127.0.0.1', 8080)

Processing command ['joinConfirm', '0'] from ('127.0.0.1', 8080)
Received join confirmation from 0



* Check member state while testing by checking contents of buffers and global variables

In [10]:
print("Group Members:\n{}".format(groupMembers))
print("\nFTq.Q:\n{}".format(FTq.Q))
print("\nClient Messages:\n{}".format(messages))
print("\nSequence Numbers:\n{}".format(seqNumsSent))
print("\nGlobal Sequence Num:\n{}".format(globalSeq))

Group Members:
{1: ('127.0.0.1', 8081), 0: ('127.0.0.1', 8080)}

FTq.Q:
{0: {'que': <queue.Queue object at 0x7f056c8b4550>, 'label': 12}}

Client Messages:
{}

Sequence Numbers:
{1: 'f043888d-6ff4-489e-b3eb-b5e116d2498d'}

Global Sequence Num:
[3]


In [9]:
FTq.Q[0]['que'].queue

deque([10, 10])

In [9]:
testBuildup.append(1)


Re requesting seguence number 2 from group member 0

Received ['seq', '88f55351-793d-4479-beaf-7baa13775618', '2'] from ('127.0.0.1', 8080)

Processing command ['seq', '88f55351-793d-4479-beaf-7baa13775618', '2'] from ('127.0.0.1', 8080)
Received sequence number 2 from 0
Sequence number 2 EQUALS 2
Running queue operation ['qPush', '0', '12']
Incrementing global seq number to 3
Message 88f55351-793d-4479-beaf-7baa13775618 processed and un buffered

